In [1]:
#3. (Back propagation) Design the simple single-layer perceptron (SLP) network for IRIS dataset.

# 1) Use python library to extract images, and separate them into test and training set.
import sklearn.datasets as dataset
import numpy as np
import matplotlib.pyplot as plt

iris = dataset.load_iris()

iris_data = iris.data
iris_target = iris.target
print('iris data shape : ', iris_data.shape)
print('iris target shape : ', iris_target.shape)
print()
def separate(data,target,testset_rate=0.2):
    rate = testset_rate

    testset_idx = np.sort( np.random.choice(len(data), round( len(data)* rate),replace=False) )
    trainingset_idx = np.sort( np.setdiff1d(np.arange(len(data)), testset_idx) )

    print('testset_idx \n')
    print(testset_idx)
    print('\n trainingset_idx \n')
    print(trainingset_idx)

    test_data = data[testset_idx]
    test_target = target[testset_idx]

    training_data = data[trainingset_idx]
    training_target = target[trainingset_idx]

    print('\n test set shape:  ', test_data.shape, test_target.shape )
    print('\n training set shape:  ', training_data.shape, training_target.shape )
    return test_data, test_target, training_data,training_target

test_data, test_target, training_data, training_target = separate(iris_data, iris_target)

iris data shape :  (150, 4)
iris target shape :  (150,)

testset_idx 

[  8   9  17  20  30  35  36  41  46  55  66  68  78  82  84  86  90  91
 105 106 108 114 115 124 132 133 134 146 147 148]

 trainingset_idx 

[  0   1   2   3   4   5   6   7  10  11  12  13  14  15  16  18  19  21
  22  23  24  25  26  27  28  29  31  32  33  34  37  38  39  40  42  43
  44  45  47  48  49  50  51  52  53  54  56  57  58  59  60  61  62  63
  64  65  67  69  70  71  72  73  74  75  76  77  79  80  81  83  85  87
  88  89  92  93  94  95  96  97  98  99 100 101 102 103 104 107 109 110
 111 112 113 116 117 118 119 120 121 122 123 125 126 127 128 129 130 131
 135 136 137 138 139 140 141 142 143 144 145 149]

 test set shape:   (30, 4) (30,)

 training set shape:   (120, 4) (120,)


In [2]:
# 2) Select any two datasets for training and test.
#random_index1 = np.random.choice(150,1)
#sample1 = iris_data[random_index1]
#label1 = iris_target[random_index1]
#random_index2 = np.random.choice(150,1)
#sample2 = iris_data[random_index2]
#label2 = iris_target[random_index2]
#print(sample1, label1)
#print(sample2, label2)

In [3]:
# 3) Design SLP network to classify them. Use the sigmoid function for activation function.
# 4) Use the back propagation method to train the network parameters. 

# sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))     # sigmoid

# sigmoid 미분
def differ_sigmoid(x):
    return sigmoid(x)*(1-sigmoid(x)) 

# MSE 미분
def differ_MSE(label, predict):
    return predict - label

class NN():
    def __init__(self, input_size, output_size, hidden_size):
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        # weights
        self.weight1 = np.random.randn(input_size, hidden_size)
        self.weight2 = np.random.randn(hidden_size, output_size)
        print('input size:    ', self.input_size)
        print('weight1 shape: ', self.weight1.shape)
        print('hidden size:   ', self.hidden_size)
        print('weight2 shape: ', self.weight2.shape)
        print('output size:   ', self.output_size)
    # Forward propagation
    def forward(self, x):
        self.hidden_in  = np.dot(x, self.weight1)
        self.hidden_out = sigmoid(self.hidden_in)
        self.output_in  = np.dot(self.hidden_out, self.weight2)
        self.output_out = sigmoid(self.output_in)
        return self.output_out
    
    def backward(self, input_, label, predict): 
        predict_error = differ_MSE(label, predict)                       # loss function differectial         
        predict_delta = predict_error * differ_sigmoid(predict)      # sigmoid differectial
        predict_delta = predict_delta
        self.weight2 -= 0.001 * np.expand_dims(self.hidden_out, axis=0).T @ np.expand_dims(predict_delta, axis=0) 
        
        hidden_error = np.dot(predict_delta, self.weight2.T)
        hidden_delta = hidden_error * differ_sigmoid(self.hidden_out)
        self.weight1 -= 0.001 * np.expand_dims(input_, axis=0).T @ np.expand_dims(hidden_delta, axis=0) 
    
input_size, output_size, hidden_size = 5, 3, 7
NN = NN(input_size, output_size, hidden_size)

input size:     5
weight1 shape:  (5, 7)
hidden size:    7
weight2 shape:  (7, 3)
output size:    3


In [4]:
# 학습 이전
def one_hot_encoding(input_, num_of_class):
    encoding = np.zeros(num_of_class)
    encoding[input_]=1
    return encoding

#input_ = np.append(sample1, 1)
#print('input:   ', input_)
#label1_ = one_hot_encoding(label1, 3)
#print('label:   ', label1)
#predict = NN.forward(input_)
#print('predict: ', predict)

In [5]:
# 5) Compute the training and test error for every epoch.

# MSE(loss function)
def MSEloss(label, predict):
    return np.mean(np.square(label - predict))

epoch = 500
for j in range(epoch):
    print('='*80)
    print('epoch: ', j+1)
    number = 0
    average_loss = 0
    for i in range(len(training_data)):                   # training set
        input_ = training_data[i]
        input_ = np.append(input_, 1)                      # input에 bias 추가
        label1_ = training_target[i]
        label1_ = one_hot_encoding(label1_,3)              # target one-hot encoding
        predict = NN.forward(input_)                       # predict
        correct = np.argmax(predict)==np.argmax(label1_)   # 정답 여부 확인
        if correct:
            number+=1
        if i%20 == 0:                                      # for visualization
            print(i,'번째 predict:', np.round_(predict, 3),'    label:', label1_, correct) 
        average_loss += MSEloss(label1_, predict)           # loss 누적
        NN.backward(input_, label1_, predict)               # back-propagation, 학습
    print()
    print('='*80) 
    print('< training set >')                               # for visualization
    print('average loss: ', 100*(average_loss/len(training_data)))
    print('accuracy: ', 100*(number/len(training_data)), '%')
    
    number = 0
    for i in range(len(test_data)):                         # test set
        input_ = test_data[i]
        input_ = np.append(input_, 1) 
        label1_ = test_target[i]
        label1_ = one_hot_encoding(label1_,3)
        predict = NN.forward(input_)
        correct = np.argmax(predict)==np.argmax(label1_)
        if correct:
            number+=1
        average_loss += MSEloss(label1_, predict)           # back-propagation 삭제됨
    print()
    print('='*80) 
    print('< test set >')
    print('average loss: ', 100*(average_loss/len(test_data)))
    print('accuracy: ', 100*(number/len(test_data)), '%')
    
    

epoch:  1
0 번째 predict: [0.085 0.878 0.754]     label: [1. 0. 0.] False
20 번째 predict: [0.087 0.875 0.754]     label: [1. 0. 0.] False
40 번째 predict: [0.087 0.865 0.735]     label: [1. 0. 0.] False
60 번째 predict: [0.099 0.633 0.522]     label: [0. 1. 0.] True
80 번째 predict: [0.095 0.721 0.575]     label: [0. 1. 0.] True
100 번째 predict: [0.1   0.619 0.502]     label: [0. 0. 1.] False

< training set >
average loss:  36.02324298684042
accuracy:  34.166666666666664 %

< test set >
average loss:  177.78114446434927
accuracy:  30.0 %
epoch:  2
0 번째 predict: [0.087 0.865 0.736]     label: [1. 0. 0.] False
20 번째 predict: [0.089 0.859 0.734]     label: [1. 0. 0.] False
40 번째 predict: [0.09  0.851 0.717]     label: [1. 0. 0.] False
60 번째 predict: [0.101 0.616 0.504]     label: [0. 1. 0.] True
80 번째 predict: [0.098 0.697 0.549]     label: [0. 1. 0.] True
100 번째 predict: [0.102 0.603 0.484]     label: [0. 0. 1.] False

< training set >
average loss:  35.28841238524432
accuracy:  34.16666666666666


< training set >
average loss:  30.992266783033916
accuracy:  34.166666666666664 %

< test set >
average loss:  153.6462768988153
accuracy:  30.0 %
epoch:  12
0 번째 predict: [0.111 0.757 0.596]     label: [1. 0. 0.] False
20 번째 predict: [0.113 0.722 0.578]     label: [1. 0. 0.] False
40 번째 predict: [0.115 0.743 0.578]     label: [1. 0. 0.] False
60 번째 predict: [0.12  0.51  0.384]     label: [0. 1. 0.] True
80 번째 predict: [0.119 0.544 0.398]     label: [0. 1. 0.] True
100 번째 predict: [0.12  0.499 0.366]     label: [0. 0. 1.] False

< training set >
average loss:  30.66330090981811
accuracy:  34.166666666666664 %

< test set >
average loss:  152.06576568130922
accuracy:  30.0 %
epoch:  13
0 번째 predict: [0.114 0.748 0.585]     label: [1. 0. 0.] False
20 번째 predict: [0.115 0.711 0.567]     label: [1. 0. 0.] False
40 번째 predict: [0.118 0.734 0.568]     label: [1. 0. 0.] False
60 번째 predict: [0.122 0.503 0.377]     label: [0. 1. 0.] True
80 번째 predict: [0.121 0.535 0.39 ]     label: [0. 1. 0

20 번째 predict: [0.136 0.623 0.487]     label: [1. 0. 0.] False
40 번째 predict: [0.143 0.661 0.496]     label: [1. 0. 0.] False
60 번째 predict: [0.136 0.459 0.338]     label: [0. 1. 0.] True
80 번째 predict: [0.135 0.478 0.345]     label: [0. 1. 0.] True
100 번째 predict: [0.135 0.45  0.324]     label: [0. 0. 1.] False

< training set >
average loss:  28.113570239344117
accuracy:  34.166666666666664 %

< test set >
average loss:  139.7782502077074
accuracy:  30.0 %
epoch:  23
0 번째 predict: [0.142 0.668 0.505]     label: [1. 0. 0.] False
20 번째 predict: [0.138 0.615 0.481]     label: [1. 0. 0.] False
40 번째 predict: [0.146 0.653 0.491]     label: [1. 0. 0.] False
60 번째 predict: [0.137 0.455 0.335]     label: [0. 1. 0.] True
80 번째 predict: [0.137 0.473 0.342]     label: [0. 1. 0.] True
100 번째 predict: [0.137 0.447 0.322]     label: [0. 0. 1.] False

< training set >
average loss:  27.9108384238792
accuracy:  34.166666666666664 %

< test set >
average loss:  138.79735990263768
accuracy:  30.0 %
ep

80 번째 predict: [0.152 0.44  0.323]     label: [0. 1. 0.] True
100 번째 predict: [0.151 0.422 0.309]     label: [0. 0. 1.] False

< training set >
average loss:  26.187136603931883
accuracy:  34.166666666666664 %

< test set >
average loss:  130.42747375057903
accuracy:  30.0 %
epoch:  34
0 번째 predict: [0.176 0.597 0.454]     label: [1. 0. 0.] False
20 번째 predict: [0.162 0.546 0.431]     label: [1. 0. 0.] False
40 번째 predict: [0.181 0.582 0.442]     label: [1. 0. 0.] False
60 번째 predict: [0.153 0.427 0.32 ]     label: [0. 1. 0.] True
80 번째 predict: [0.153 0.438 0.322]     label: [0. 1. 0.] True
100 번째 predict: [0.152 0.42  0.308]     label: [0. 0. 1.] False

< training set >
average loss:  26.037340368452437
accuracy:  34.166666666666664 %

< test set >
average loss:  129.69751136451134
accuracy:  30.0 %
epoch:  35
0 번째 predict: [0.179 0.591 0.45 ]     label: [1. 0. 0.] False
20 번째 predict: [0.164 0.541 0.428]     label: [1. 0. 0.] False
40 번째 predict: [0.184 0.577 0.439]     label: [1. 0

< training set >
average loss:  24.05613859197139
accuracy:  34.166666666666664 %

< test set >
average loss:  120.00920800926794
accuracy:  30.0 %
epoch:  50
0 번째 predict: [0.234 0.515 0.412]     label: [1. 0. 0.] False
20 번째 predict: [0.197 0.483 0.398]     label: [1. 0. 0.] False
40 번째 predict: [0.238 0.503 0.403]     label: [1. 0. 0.] False
60 번째 predict: [0.172 0.407 0.321]     label: [0. 1. 0.] True
80 번째 predict: [0.173 0.411 0.316]     label: [0. 1. 0.] True
100 번째 predict: [0.171 0.4   0.309]     label: [0. 0. 1.] False

< training set >
average loss:  23.936249597977753
accuracy:  34.166666666666664 %

< test set >
average loss:  119.42117661936011
accuracy:  30.0 %
epoch:  51
0 번째 predict: [0.238 0.51  0.41 ]     label: [1. 0. 0.] False
20 번째 predict: [0.199 0.48  0.396]     label: [1. 0. 0.] False
40 번째 predict: [0.242 0.499 0.402]     label: [1. 0. 0.] False
60 번째 predict: [0.173 0.406 0.322]     label: [0. 1. 0.] True
80 번째 predict: [0.174 0.41  0.316]     label: [0. 1. 0

...

100 번째 predict: [0.108 0.437 0.45 ]     label: [0. 0. 1.] True

< training set >
average loss:  12.305483750916183
accuracy:  91.66666666666666 %

< test set >
average loss:  60.68032104101364
accuracy:  96.66666666666667 %
epoch:  443
0 번째 predict: [0.638 0.202 0.163]     label: [1. 0. 0.] True
20 번째 predict: [0.551 0.212 0.153]     label: [1. 0. 0.] True
40 번째 predict: [0.639 0.202 0.163]     label: [1. 0. 0.] True
60 번째 predict: [0.107 0.445 0.475]     label: [0. 1. 0.] False
80 번째 predict: [0.419 0.258 0.176]     label: [0. 1. 0.] False
100 번째 predict: [0.108 0.437 0.45 ]     label: [0. 0. 1.] True

< training set >
average loss:  12.300400917217267
accuracy:  91.66666666666666 %

< test set >
average loss:  60.65317513585222
accuracy:  96.66666666666667 %
epoch:  444
0 번째 predict: [0.638 0.202 0.162]     label: [1. 0. 0.] True
20 번째 predict: [0.551 0.212 0.153]     label: [1. 0. 0.] True
40 번째 predict: [0.639 0.202 0.163]     label: [1. 0. 0.] True
60 번째 predict: [0.107 0.445 0.47

20 번째 predict: [0.552 0.211 0.151]     label: [1. 0. 0.] True
40 번째 predict: [0.641 0.202 0.161]     label: [1. 0. 0.] True
60 번째 predict: [0.105 0.446 0.475]     label: [0. 1. 0.] False
80 번째 predict: [0.422 0.256 0.174]     label: [0. 1. 0.] False
100 번째 predict: [0.106 0.438 0.451]     label: [0. 0. 1.] True

< training set >
average loss:  12.250320527712315
accuracy:  91.66666666666666 %

< test set >
average loss:  60.38580006760274
accuracy:  100.0 %
epoch:  454
0 번째 predict: [0.64  0.202 0.16 ]     label: [1. 0. 0.] True
20 번째 predict: [0.552 0.211 0.151]     label: [1. 0. 0.] True
40 번째 predict: [0.642 0.202 0.16 ]     label: [1. 0. 0.] True
60 번째 predict: [0.105 0.446 0.475]     label: [0. 1. 0.] False
80 번째 predict: [0.423 0.256 0.174]     label: [0. 1. 0.] False
100 번째 predict: [0.106 0.438 0.451]     label: [0. 0. 1.] True

< training set >
average loss:  12.245386273023268
accuracy:  91.66666666666666 %

< test set >
average loss:  60.359464984658416
accuracy:  100.0 %
ep

< test set >
average loss:  60.125662224818456
accuracy:  100.0 %
epoch:  464
0 번째 predict: [0.643 0.201 0.158]     label: [1. 0. 0.] True
20 번째 predict: [0.554 0.211 0.149]     label: [1. 0. 0.] True
40 번째 predict: [0.644 0.201 0.158]     label: [1. 0. 0.] True
60 번째 predict: [0.104 0.447 0.476]     label: [0. 1. 0.] False
80 번째 predict: [0.426 0.255 0.172]     label: [0. 1. 0.] False
100 번째 predict: [0.105 0.439 0.451]     label: [0. 0. 1.] True

< training set >
average loss:  12.19676402344299
accuracy:  91.66666666666666 %

< test set >
average loss:  60.100036390571475
accuracy:  100.0 %
epoch:  465
0 번째 predict: [0.643 0.201 0.158]     label: [1. 0. 0.] True
20 번째 predict: [0.554 0.211 0.149]     label: [1. 0. 0.] True
40 번째 predict: [0.644 0.201 0.158]     label: [1. 0. 0.] True
60 번째 predict: [0.103 0.447 0.476]     label: [0. 1. 0.] False
80 번째 predict: [0.426 0.255 0.171]     label: [0. 1. 0.] False
100 번째 predict: [0.104 0.439 0.452]     label: [0. 0. 1.] True

< training s

40 번째 predict: [0.646 0.201 0.156]     label: [1. 0. 0.] True
60 번째 predict: [0.102 0.449 0.477]     label: [0. 1. 0.] False
80 번째 predict: [0.429 0.254 0.169]     label: [0. 1. 0.] False
100 번째 predict: [0.103 0.44  0.452]     label: [0. 0. 1.] True

< training set >
average loss:  12.144754156341127
accuracy:  91.66666666666666 %

< test set >
average loss:  59.822685223735164
accuracy:  100.0 %
epoch:  476
0 번째 predict: [0.645 0.201 0.156]     label: [1. 0. 0.] True
20 번째 predict: [0.556 0.211 0.146]     label: [1. 0. 0.] True
40 번째 predict: [0.646 0.201 0.156]     label: [1. 0. 0.] True
60 번째 predict: [0.102 0.449 0.477]     label: [0. 1. 0.] False
80 번째 predict: [0.43  0.254 0.169]     label: [0. 1. 0.] False
100 번째 predict: [0.103 0.441 0.452]     label: [0. 0. 1.] True

< training set >
average loss:  12.140100641032276
accuracy:  91.66666666666666 %

< test set >
average loss:  59.79787717049728
accuracy:  100.0 %
epoch:  477
0 번째 predict: [0.646 0.201 0.155]     label: [1. 0. 


< training set >
average loss:  12.07174436966603
accuracy:  91.66666666666666 %

< test set >
average loss:  59.433612654213675
accuracy:  100.0 %
epoch:  492
0 번째 predict: [0.649 0.2   0.152]     label: [1. 0. 0.] True
20 번째 predict: [0.558 0.211 0.144]     label: [1. 0. 0.] True
40 번째 predict: [0.65  0.2   0.153]     label: [1. 0. 0.] True
60 번째 predict: [0.099 0.45  0.478]     label: [0. 1. 0.] False
80 번째 predict: [0.435 0.253 0.166]     label: [0. 1. 0.] False
100 번째 predict: [0.1   0.442 0.453]     label: [0. 0. 1.] True

< training set >
average loss:  12.06728179982523
accuracy:  91.66666666666666 %

< test set >
average loss:  59.40984164174872
accuracy:  100.0 %
epoch:  493
0 번째 predict: [0.649 0.2   0.152]     label: [1. 0. 0.] True
20 번째 predict: [0.558 0.211 0.143]     label: [1. 0. 0.] True
40 번째 predict: [0.65  0.2   0.152]     label: [1. 0. 0.] True
60 번째 predict: [0.099 0.45  0.479]     label: [0. 1. 0.] False
80 번째 predict: [0.435 0.253 0.165]     label: [0. 1. 0.] 